In [1]:
import tensorflow as tf
from transformers import *

C:\Users\liuzhehao\AppData\Local\Continuum\anaconda3\envs\dl\lib\site-packages\ipykernel\parentpoller.py:116: UserWarning: Parent poll failed.  If the frontend dies,
                the kernel may be left running.  Please let us know
                about your system (bitness, Python, etc.) at
                ipython-dev@scipy.org
  ipython-dev@scipy.org""")
I1113 16:21:38.630255 61452 file_utils.py:32] TensorFlow version 2.0.0 available.
I1113 16:21:39.234316 61452 file_utils.py:39] PyTorch version 1.0.0 available.
I1113 16:21:40.394432 61452 modeling_xlnet.py:194] Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex .


In [2]:
# try out bert
model_class, tokenizer_class, pretrained_weights = \
    TFBertForSequenceClassification, BertTokenizer, 'bert-base-chinese'
tokenizer = tokenizer_class.from_pretrained(pretrained_weights)

I1113 16:21:47.195111 61452 tokenization_utils.py:374] loading file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-chinese-vocab.txt from cache at C:\Users\liuzhehao\.cache\torch\transformers\8a0c070123c1f794c42a29c6904beb7c1b8715741e235bee04aca2c7636fc83f.9b42061518a39ca00b8b52059fd2bede8daa613f8a8671500e518a8c29de8c00


In [3]:
model = model_class.from_pretrained('bert-base-chinese')

I1113 16:21:51.274519 61452 configuration_utils.py:151] loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-chinese-config.json from cache at C:\Users\liuzhehao\.cache\torch\transformers\8a3b1cfe5da58286e12a0f5d7d182b8d6eca88c08e26c332ee3817548cf7e60a.0c16faba8be66db3f02805c912e4cf94d3c9cffc1f12fa1a39906f9270f76d33
I1113 16:21:51.283520 61452 configuration_utils.py:168] Model config {
  "attention_probs_dropout_prob": 0.1,
  "directionality": "bidi",
  "finetuning_task": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "num_labels": 2,
  "output_attentions": false,
  "output_hidden_states": false,
  "output_past": true,
  "pooler_fc_size": 768,
  "pooler_num_attention_heads": 12,
  "pooler_num_fc_layers": 3,
  "pooler_size_per_head": 128,
  "p

In [116]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

df = pd.read_csv('https://raw.githubusercontent.com/SophonPlus/ChineseNlpCorpus/master/datasets/ChnSentiCorp_htl_all/ChnSentiCorp_htl_all.csv')

In [ ]:
X, y = [], []

for label, text in zip(df['label'], df['review']):
    try:
        X.append(tokenizer.encode(text, max_length=500))
        y.append(label)
    except:
        pass
    
X = tf.ragged.constant(X)
X = X.to_tensor()
y = np.array(y)

In [179]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X.numpy(), y, train_size=0.8, stratify=y, random_state=1024)
y_train = keras.utils.to_categorical(y_train)
y_test = keras.utils.to_categorical(y_test)

train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train))
train_dataset.batch(32)

test_dataset = tf.data.Dataset.from_tensor_slices((X_test, y_test))

In [181]:
# finetuning
from tensorflow import keras

for layer in model.layers[:-1]:
    layer.trainable = False
model.layers[-1].trainable = True

model.compile(optimizer=keras.optimizers.Adam(),
             loss=keras.losses.CategoricalCrossentropy())

In [ ]:
model.fit(X_train, y_train, epochs=1)

Train on 6212 samples
